## Exercise 8.1

**Question 1:**

In an autoencoder, the number of nodes in the input layer is _____ the number of nodes in the output payer:

1. Smaller than
2. Greater than
3. Smaller than or equal to
4. None of the above

**Answer: 4**

**Question 2:**

An autoencoder is a type of:

1. Reinforcement learning algorithm
2. Supervised learning algorithm
3. Unsupervised learning algorithm
4. Semi-supervised learning algorith

**Answer: 3**

**Question 3:**

Which of the following is an application of autoencoders:

1. Image denoising
2. Recommendation System
3. Dimensionality reduction
4. All of the above

**Answer: 4**


## Exercise 8.2 

Using breast cancer dataset as follows, perform anomaly detection on the patients who have breast cancer. Divide the data into training and test sets. Learn the threshold for detecting anomaly on the training and use the threshold to make predictions on the complete test set. The dataset can be downloaded with the following script:

In [0]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

bc_data = pd.DataFrame(cancer.data, columns=cancer.feature_names)
bc_data['Target'] = pd.Series(cancer.target)

**Solution**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
bc_data = pd.DataFrame(cancer.data, columns=cancer.feature_names)
bc_data['Target'] = pd.Series(cancer.target)
bc_data.head()
bc_data.shape
bc_data['Target'].value_counts()
X =  bc_data.drop(['Target'], axis=1)
y = bc_data[['Target']]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
print(X_train.shape)
print(X_test.shape)
X_train_T = X_train.copy()
X_train_T['Target'] = y_train
X_train_T = X_train_T[X_train_T['Target']==0]
X_train_T = X_train_T.drop('Target', axis=1)
X_train_T = np.asarray(X_train_T).astype(np.float32)

X_test_T = X_test.copy()
X_test_T['Target'] = y_test
X_test_T = X_test_T[X_test_T['Target']==0]
X_test_T = X_test_T.drop('Target', axis=1)
X_test_T = np.asarray(X_test_T).astype(np.float32)
import pandas as pd
import numpy as np
import re


from numpy import array
from tensorflow.keras.preprocessing.text import one_hot

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, LSTM, Dropout, Dense, Flatten, Input,  Embedding, Conv1D, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from sklearn.metrics import recall_score, classification_report, auc, roc_curve, accuracy_score

epochs = 800
batch_size = 2
input_dim = X_train_T.shape[1]
encoding_dim = 16
hidden_layers_nodes = 8

learning_rate = 1e-3


input_layer = Input(shape=(input_dim, ))
## Encoder Layers
encoder_layer1 = Dense(encoding_dim, activation="relu")(input_layer)
encoder_layer2 = Dense(hidden_layers_nodes, activation="relu")(encoder_layer1)

## Decoder Layers
decoder_layer1 = Dense(hidden_layers_nodes, activation="relu")(encoder_layer2)
decoder_layer2 = Dense(encoding_dim, activation="relu")(decoder_layer1)
decoder_output = Dense(input_dim, activation="linear")(decoder_layer2)

autoencoder = Model(inputs=input_layer, outputs=decoder_output)
autoencoder.summary()
autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')

history = autoencoder.fit(X_train_T, X_train_T,
                    epochs=epochs,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_test_T, X_test_T),
                    verbose=1)
X_test_pred = autoencoder.predict(X_test)
print(X_test_pred.shape)
mean_seq_error = np.mean(np.power(X_test - X_test_pred, 2), axis=1)
mean_seq_error
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, mean_seq_error)
print('thresholds', np.mean(thresholds))
auc(false_positive_rate,  true_positive_rate)
final_threshold =  np.mean(thresholds)
accuracy_score(y_test, [1 if i > final_threshold else 0 for i in mean_seq_error])

(398, 30)
(171, 30)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
dense (Dense)                (None, 16)                496       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_3 (Dense)              (None, 16)                144       
_________________________________________________________________
dense_4 (Dense)              (None, 30)                510       
Total params: 1,358
Trainable params: 1,358
Non-trainable params: 0
_______________________________________

0.30409356725146197